In [1]:
import pandas as pd

import torch.nn.utils.prune as prune

import torch

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from typing import Callable, Tuple
from pathlib import Path

from dataclasses import dataclass, asdict

### Load the data

In [2]:
# load FashionMNIST data
transform = transforms.Compose([transforms.ToTensor()])

# split into validation and train datasets
train_ds = datasets.FashionMNIST("data", train=True, transform=transform, download=True)
train_ds, valid_ds = random_split(train_ds, [0.8, 0.2])

test_ds = datasets.FashionMNIST("data", train=False, transform=transform, download=True)

100%|██████████| 26421880/26421880 [00:01<00:00, 15132572.47it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 609830.01it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 7979796.33it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 10151517.16it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



## Define the model architecture

In [3]:
# Define a simple CNN model
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            1, 6, kernel_size=5, stride=1, padding=2
        )  # 28*28->32*32-->28*28
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.flatten1 = nn.Flatten()
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.flatten1(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Define utilities for training and testing

In [4]:
class EarlyStopper:
    def __init__(self, patience: int = 1, min_delta: int = 0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float("inf")

    def early_stop(self, validation_loss: float) -> bool:
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [5]:
# Define a function to train the model
def fit(
    model: nn.Module,
    train_dl,
    valid_dl,
    optimizer: optim.Optimizer,
    loss_function: Callable,
    epochs: int,
    early_stopper: EarlyStopper | None = None,
    device: torch.device = torch.device("cpu"),
) -> Tuple[float, float]:
    valid_loss = 0
    valid_accuracy = 0

    for epoch in range(epochs):
        model.train()
        for X, y in train_dl:
            X, y = X.to(device), y.to(device)

            # Compute prediction error
            pred = model(X)
            train_loss = loss_function(pred, y)

            # Backpropagation
            train_loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        model.eval()
        valid_loss = 0
        valid_accuracy = 0
        with torch.no_grad():
            for X, y in valid_dl:
                X, y = X.to(device), y.to(device)

                # Compute prediction error
                pred = model(X)
                valid_loss += loss_function(pred, y)

                # Compute accuracy
                valid_accuracy += (pred.argmax(1) == y).float().mean()

        valid_loss /= len(valid_dl)
        valid_accuracy /= len(valid_dl)

        print(
            f"Epoch #{epoch + 1}:\t validation loss: {valid_loss:.4f}\t validation accuracy: {valid_accuracy:.4f}"
        )

        if early_stopper is not None and early_stopper.early_stop(valid_loss):
            print("Early stopping")
            return (valid_loss, valid_accuracy)

    return (valid_loss, valid_accuracy)

In [6]:
# Define a function to test the model
def test(
    model: nn.Module,
    test_dl,
    loss_function: Callable,
    device: torch.device = torch.device("cpu"),
) -> Tuple[float, float]:
    size = len(test_dl.dataset)
    num_batches = len(test_dl)
    model.eval()

    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in test_dl:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_function(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    accuracy = (correct / size) * 100

    return (test_loss, accuracy)

## Training Phase

In [7]:
# define the constants
BATCH_SIZE: int = 32
LEARNING_RATE: float = 0.01
EPOCHS: int = 10
MOMENTUM: float = 0.9

In [8]:
# Get cpu, gpu or mps device for training.
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using {torch.cuda.get_device_name(torch.cuda.current_device())}")

Using NVIDIA GeForce GTX 1660 Ti


In [9]:
base_model = LeNet().to(device)

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
early_stopper = EarlyStopper(patience=3, min_delta=0)
optimizer = optim.SGD(base_model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

# create the data loaders
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

### Training loop

In [10]:
valid_loss, valid_accuracy = fit(
    base_model,
    train_dl=train_loader,
    valid_dl=validation_loader,
    optimizer=optimizer,
    loss_function=loss_fn,
    epochs=EPOCHS,
    device=device,
)

/home/bubuss/miniforge3/envs/ml/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1699449183005/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch #1:	 validation loss: 0.5069	 validation accuracy: 0.8076
Epoch #2:	 validation loss: 0.3645	 validation accuracy: 0.8656
Epoch #3:	 validation loss: 0.3177	 validation accuracy: 0.8787
Epoch #4:	 validation loss: 0.3086	 validation accuracy: 0.8840
Epoch #5:	 validation loss: 0.3039	 validation accuracy: 0.8870
Epoch #6:	 validation loss: 0.3007	 validation accuracy: 0.8888
Epoch #7:	 validation loss: 0.2786	 validation accuracy: 0.8977
Epoch #8:	 validation loss: 0.2996	 validation accuracy: 0.8867
Epoch #9:	 validation loss: 0.2720	 validation accuracy: 0.8986
Epoch #10:	 validation loss: 0.2699	 validation accuracy: 0.9006


In [11]:
test_loss, accuracy = test(
    base_model, test_dl=test_loader, loss_function=loss_fn, device=device
)
print(f"Test Error: \n Accuracy: {accuracy:>0.1f}%, Avg loss: {test_loss:>8f} \n")

Test Error: 
 Accuracy: 89.6%, Avg loss: 0.290767 



In [12]:
for i, module in enumerate(base_model.modules()):
    print(i, module)

0 LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (flatten1): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
1 Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
2 AvgPool2d(kernel_size=2, stride=2, padding=0)
3 Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
4 AvgPool2d(kernel_size=2, stride=2, padding=0)
5 Flatten(start_dim=1, end_dim=-1)
6 Linear(in_features=400, out_features=120, bias=True)
7 Linear(in_features=120, out_features=84, bias=True)
8 Linear(in_features=84, out_features=10, bias=True)


Save the model

In [13]:
torch.save(base_model.state_dict(), f"models/{type(base_model).__name__}_fmnist.pth")

## Pruning Phase

### One shot pruning

In [14]:
def get_model_sparsity(m: nn.Module) -> float:
    """Get the sparsity of the model

    Args:
        model (nn.Module): The model to get the sparsity of

    Returns:
        float: percentage of weights that are zero
    """

    total_weights = 0
    total_zero_weights = 0
    for _, module in m.named_children():
        for param_name, param in module.named_parameters():
            if "weight" in param_name:
                total_weights += float(param.nelement())
                total_zero_weights += float(torch.sum(param == 0))

    sparsity = 100.0 * total_zero_weights / total_weights
    return sparsity


def get_layers_sparsity(model: nn.Module) -> list[tuple[str, float]]:
    """Get the sparsity of each layer in the model

    Args:
        model (nn.Module): The model to get the sparsity of

    Returns:
        list[tuple[str, float]]: List of tuples containing the layer name and the sparsity
    """

    layers_sparsity = []
    for layer_name, module in model.named_children():
        for param_name, param in module.named_parameters():
            if "weight" in param_name:
                layer_sparsity = (
                    100.0 * float(torch.sum(param == 0)) / float(param.nelement())
                )
                layers_sparsity.append((layer_name, layer_sparsity))

    return layers_sparsity

In [15]:
def get_parameters_to_prune(model: nn.Module) -> list[nn.Parameter]:
    return [
        (module, "weight")
        for module in model.modules()
        if isinstance(module, nn.Conv2d | nn.Linear)
    ]

In [16]:
PRUNING_VALUES = [0.2, 0.4, 0.6, 0.8, 0.9, 0.95]
PRUNING_METHODS = {
    "RandomUnstructured": prune.RandomUnstructured,
    "L1Unstructured": prune.L1Unstructured,
}

In [30]:
@dataclass
class PruningResult:
    method: str
    pruning_rate: float
    val_accuracy: float
    val_loss: float

    def __str__(self):
        return f"""Method:{self.method} with pruning rate: {self.pruning_rate}
    Validation accuracy: {round(self.val_accuracy, 2)}\tValidation loss: {round(self.val_loss, 2)}"""

In [18]:
results = []
for pruning_rate in PRUNING_VALUES:
    for method_name, method in PRUNING_METHODS.items():
        # load the model
        temp_model = LeNet().to(device)
        temp_model.load_state_dict(torch.load("models/LeNet_fmnist.pth"))
        model_parameters = get_parameters_to_prune(temp_model)

        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.SGD(
            temp_model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM
        )

        # create the data loaders
        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
        validation_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE)
        test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

        print(f"Pre-pruning sparsity: {100 - get_model_sparsity(temp_model):.2f}%")

        # prune the model
        prune.global_unstructured(
            parameters=model_parameters,
            pruning_method=method,
            amount=pruning_rate,
        )

        print(f"Pruning rate: {pruning_rate}, method: {method_name}")

        val_loss, val_accuracy = fit(
            model=temp_model,
            train_dl=train_loader,
            valid_dl=validation_loader,
            optimizer=optimizer,
            loss_function=loss_fn,
            epochs=3,
            device=device,
        )

        results.append(
            PruningResult(
                method=method_name,
                pruning_rate=pruning_rate,
                val_accuracy=val_accuracy,
                val_loss=val_loss,
            )
        )

        for module, name in model_parameters:
            prune.remove(module, name)

        print(f"Post-pruning sparsity: {100 - get_model_sparsity(temp_model):.2f}%")
        torch.save(
            temp_model.state_dict(),
            f"models/{type(temp_model).__name__}_pruned_{pruning_rate}_{method_name}.pth",
        )

Pre-pruning sparsity: 100.00%
Pruning rate: 0.2, method: RandomUnstructured


Epoch #1:	 validation loss: 0.2697	 validation accuracy: 0.9004
Epoch #2:	 validation loss: 0.2496	 validation accuracy: 0.9117
Epoch #3:	 validation loss: 0.2576	 validation accuracy: 0.9073
Post-pruning sparsity: 80.00%
Pre-pruning sparsity: 100.00%
Pruning rate: 0.2, method: L1Unstructured
Epoch #1:	 validation loss: 0.2599	 validation accuracy: 0.9056
Epoch #2:	 validation loss: 0.2464	 validation accuracy: 0.9109
Epoch #3:	 validation loss: 0.2606	 validation accuracy: 0.9097
Post-pruning sparsity: 80.00%
Pre-pruning sparsity: 100.00%
Pruning rate: 0.4, method: RandomUnstructured
Epoch #1:	 validation loss: 0.3030	 validation accuracy: 0.8860
Epoch #2:	 validation loss: 0.2604	 validation accuracy: 0.9017
Epoch #3:	 validation loss: 0.2669	 validation accuracy: 0.8997
Post-pruning sparsity: 60.00%
Pre-pruning sparsity: 100.00%
Pruning rate: 0.4, method: L1Unstructured
Epoch #1:	 validation loss: 0.2435	 validation accuracy: 0.9103
Epoch #2:	 validation loss: 0.2744	 validation acc

In [47]:
print("Method\t Pruning Rate\t Validation Loss\t Validation Accuracy")

results_df: pd.DataFrame = pd.DataFrame([asdict(result) for result in results])
results_df["val_accuracy"] = results_df["val_accuracy"].apply(
    lambda x: round(x.item(), 2)
)
results_df["val_loss"] = results_df["val_loss"].apply(lambda x: round(x.item(), 2))
print(results_df)

Method	 Pruning Rate	 Validation Loss	 Validation Accuracy
                method  pruning_rate  val_accuracy  val_loss
0   RandomUnstructured          0.20          0.91      0.26
1       L1Unstructured          0.20          0.91      0.26
2   RandomUnstructured          0.40          0.90      0.27
3       L1Unstructured          0.40          0.91      0.24
4   RandomUnstructured          0.60          0.89      0.29
5       L1Unstructured          0.60          0.91      0.25
6   RandomUnstructured          0.80          0.85      0.41
7       L1Unstructured          0.80          0.91      0.25
8   RandomUnstructured          0.90          0.81      0.53
9       L1Unstructured          0.90          0.90      0.27
10  RandomUnstructured          0.95          0.75      0.69
11      L1Unstructured          0.95          0.89      0.29


### Iterative pruning

In [20]:
RANGE: int = 20
ITER_PRUNING_AMOUNT: float = 0.01
for method_name, method in PRUNING_METHODS.items():
    print(
        f"Iterative pruning using {method_name} for {RANGE} iterations with amount {ITER_PRUNING_AMOUNT}"
    )

    iterative_model = LeNet().to(device)
    iterative_model.load_state_dict(torch.load("models/LeNet_fmnist.pth"))

    iterative_model_parameters = get_parameters_to_prune(iterative_model)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(temp_model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

    # create the data loaders
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    validation_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

    for iteration in range(RANGE):
        prune.global_unstructured(
            parameters=iterative_model_parameters,
            pruning_method=method,
            amount=ITER_PRUNING_AMOUNT,
        )

        val_loss, val_accuracy = fit(
            model=iterative_model,
            train_dl=train_loader,
            valid_dl=validation_loader,
            optimizer=optimizer,
            loss_function=loss_fn,
            epochs=1,
            device=device,
        )

        print(
            f"Iteration #{iteration + 1}:\t validation loss: {val_loss:.4f}\t validation accuracy: {val_accuracy:.4f}"
        )

    for module, name in iterative_model_parameters:
        prune.remove(module, name)

    torch.save(
        iterative_model.state_dict(),
        f"models/{type(iterative_model).__name__}_iterative_pruned_0.{RANGE}_{method_name}.pth",
    )

Iterative pruning using RandomUnstructured for 20 iterations with amount 0.01
Epoch #1:	 validation loss: 0.2858	 validation accuracy: 0.8955
Iteration #1:	 validation loss: 0.2858	 validation accuracy: 0.8955
Epoch #1:	 validation loss: 0.3293	 validation accuracy: 0.8765
Iteration #2:	 validation loss: 0.3293	 validation accuracy: 0.8765
Epoch #1:	 validation loss: 0.3751	 validation accuracy: 0.8564
Iteration #3:	 validation loss: 0.3751	 validation accuracy: 0.8564
Epoch #1:	 validation loss: 0.3741	 validation accuracy: 0.8592
Iteration #4:	 validation loss: 0.3741	 validation accuracy: 0.8592
Epoch #1:	 validation loss: 0.4481	 validation accuracy: 0.8353
Iteration #5:	 validation loss: 0.4481	 validation accuracy: 0.8353
Epoch #1:	 validation loss: 0.4704	 validation accuracy: 0.8267
Iteration #6:	 validation loss: 0.4704	 validation accuracy: 0.8267
Epoch #1:	 validation loss: 0.5506	 validation accuracy: 0.8055
Iteration #7:	 validation loss: 0.5506	 validation accuracy: 0.805

## Load and test the models

In [21]:
models = []
for file in Path("models").glob("*.pth"):
    model = LeNet().to(device)
    temp = torch.load(file)
    model.load_state_dict(temp)
    print(f"Loaded {file.stem}")
    models.append((file.stem, model))

Loaded LeNet_pruned_0.2_L1Unstructured
Loaded LeNet_pruned_0.8_RandomUnstructured
Loaded LeNet_pruned_0.2_RandomUnstructured
Loaded LeNet_iterative_pruned_0.20_RandomUnstructured
Loaded LeNet_fmnist
Loaded LeNet_pruned_0.8_L1Unstructured
Loaded LeNet_pruned_0.4_RandomUnstructured
Loaded LeNet_pruned_0.9_RandomUnstructured
Loaded LeNet_iterative_pruned_0.20_L1Unstructured
Loaded LeNet_pruned_0.95_L1Unstructured
Loaded LeNet_pruned_0.4_L1Unstructured
Loaded LeNet_pruned_0.6_L1Unstructured
Loaded LeNet_pruned_0.9_L1Unstructured
Loaded LeNet_pruned_0.95_RandomUnstructured
Loaded LeNet_pruned_0.6_RandomUnstructured


In [22]:
# create the data loaders
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)
loss_fn = nn.CrossEntropyLoss()

In [48]:
for name, model in sorted(models, key=lambda x: x[0]):
    test_loss, accuracy = test(
        model, test_dl=test_loader, loss_function=loss_fn, device=device
    )
    print(f"Model {name}")
    print(f"Test Error: \n Accuracy: {accuracy:>0.1f}%, Avg loss: {test_loss:>2f} \n")

Model LeNet_fmnist
Test Error: 
 Accuracy: 89.6%, Avg loss: 0.290767 

Model LeNet_iterative_pruned_0.20_L1Unstructured
Test Error: 
 Accuracy: 89.5%, Avg loss: 0.292396 

Model LeNet_iterative_pruned_0.20_RandomUnstructured
Test Error: 
 Accuracy: 55.9%, Avg loss: 1.127697 

Model LeNet_pruned_0.2_L1Unstructured
Test Error: 
 Accuracy: 90.2%, Avg loss: 0.285993 

Model LeNet_pruned_0.2_RandomUnstructured
Test Error: 
 Accuracy: 90.0%, Avg loss: 0.277502 

Model LeNet_pruned_0.4_L1Unstructured
Test Error: 
 Accuracy: 90.6%, Avg loss: 0.264413 

Model LeNet_pruned_0.4_RandomUnstructured
Test Error: 
 Accuracy: 89.6%, Avg loss: 0.289463 

Model LeNet_pruned_0.6_L1Unstructured
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.280371 

Model LeNet_pruned_0.6_RandomUnstructured
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.313334 

Model LeNet_pruned_0.8_L1Unstructured
Test Error: 
 Accuracy: 90.3%, Avg loss: 0.272401 

Model LeNet_pruned_0.8_RandomUnstructured
Test Error: 
 Accuracy: 84.0%, Avg loss

### Print the weights of the models

In [49]:
for name, model in sorted(models, key=lambda x: x[0]):
    print(f"Model {name}")
    print("#" * 10)

    print("Layer Sparsity:")
    for layer_name, layer_sparsity in get_layers_sparsity(model):
        print(f"{layer_name}: {100 - layer_sparsity:.2f}%")
    print(f"Model Sparsity: {100 - get_model_sparsity(model)}%")

Model LeNet_fmnist
##########
Layer Sparsity:
conv1: 100.00%
conv2: 100.00%
fc1: 100.00%
fc2: 100.00%
fc3: 100.00%
Model Sparsity: 100.0%
Model LeNet_iterative_pruned_0.20_L1Unstructured
##########
Layer Sparsity:
conv1: 98.67%
conv2: 91.67%
fc1: 79.75%
fc2: 87.81%
fc3: 95.24%
Model Sparsity: 81.79111761835041%
Model LeNet_iterative_pruned_0.20_RandomUnstructured
##########
Layer Sparsity:
conv1: 76.67%
conv2: 80.96%
fc1: 81.93%
fc2: 81.60%
fc3: 79.40%
Model Sparsity: 81.79111761835041%
Model LeNet_pruned_0.2_L1Unstructured
##########
Layer Sparsity:
conv1: 98.67%
conv2: 91.00%
fc1: 77.73%
fc2: 86.68%
fc3: 94.76%
Model Sparsity: 80.0%
Model LeNet_pruned_0.2_RandomUnstructured
##########
Layer Sparsity:
conv1: 83.33%
conv2: 80.79%
fc1: 79.99%
fc2: 79.53%
fc3: 83.33%
Model Sparsity: 80.0%
Model LeNet_pruned_0.4_L1Unstructured
##########
Layer Sparsity:
conv1: 96.67%
conv2: 81.46%
fc1: 55.54%
fc2: 73.13%
fc3: 89.64%
Model Sparsity: 60.0%
Model LeNet_pruned_0.4_RandomUnstructured
#########